In [20]:
import pandas as pd

In [4]:
import logging

from download_data import download_genie
from process_data import process_genie_data
import dataset
from dataset import create_dataset
from ml_model import train_model

from utils import read_yaml

logging.basicConfig()
logging.root.setLevel(logging.INFO)
log = logging.getLogger('Hack4NF')


log.info('Reading config files')
# Read configuration files
config_path = "config/resources.yaml"
config = read_yaml(config_path)
genie = config["genie"]


# Read synapses credentials
secrets_path = ".secrets/synapses_credentials.yaml"
credentials = read_yaml(secrets_path)

INFO:Hack4NF:Reading config files


In [5]:
genie_processed = {}
genie_processed = dataset.load_genie_processed_data(genie)

In [6]:
len(genie_processed)

3

In [13]:
genie_processed.keys()

dict_keys(['clinical_patient', 'clinical_sample', 'mutations_extended'])

In [43]:
genie_processed["clinical_sample"].head()

,PATIENT_ID,SAMPLE_ID,AGE_AT_SEQ_REPORT,ONCOTREE_CODE,SAMPLE_TYPE,SEQ_ASSAY_ID,CANCER_TYPE,CANCER_TYPE_DETAILED,AGE_AT_SEQ_REPORT_DAYS,SAMPLE_TYPE_DETAILED,SEQ_YEAR,SAMPLE_CLASS
0,GENIE-VICC-101416,GENIE-VICC-101416-unk-1,52.0,MAAP,Primary,VICC-01-T7,Appendiceal Cancer,Mucinous Adenocarcinoma of the Appendix,19247.0,Primary tumor,2014.0,Tumor
1,GENIE-VICC-102225,GENIE-VICC-102225-unk-1,31.0,READ,Metastasis,VICC-01-T7,Colorectal Cancer,Rectal Adenocarcinoma,11589.0,Metastasis site unspecified,2014.0,Tumor
2,GENIE-VICC-102424,GENIE-VICC-102424-unk-1,63.0,SCCNOS,Metastasis,VICC-01-T7,Cancer of Unknown Primary,"Squamous Cell Carcinoma, NOS",23302.0,Metastasis site unspecified,2015.0,Tumor
3,GENIE-VICC-102966,GENIE-VICC-102966-unk-1,64.0,LUAD,Metastasis,VICC-01-T7,Non-Small Cell Lung Cancer,Lung Adenocarcinoma,23390.0,Metastasis site unspecified,2015.0,Tumor
4,GENIE-VICC-103244,GENIE-VICC-103244-unk-1,50.0,IDC,Primary,VICC-01-T5A,Breast Cancer,Breast Invasive Ductal Carcinoma,18284.0,Primary tumor,2014.0,Tumor


In [ ]:
df

In [46]:
df_cs = genie_processed["clinical_sample"][["PATIENT_ID", "CANCER_TYPE"]].copy()
df_cs = pd.get_dummies(df_cs.set_index('PATIENT_ID'), prefix='', prefix_sep='')
df_cs.reset_index(inplace=True)
df_cs = df_cs.groupby("PATIENT_ID").max()
df_cs.reset_index(inplace=True)

In [47]:
df_cs.head()

,PATIENT_ID,Adenocarcinoma In Situ,Adrenal Gland Cancer,Adrenocortical Adenoma,Adrenocortical Carcinoma,Ampullary Cancer,Anal Cancer,Angiomatoid Fibrous Histiocytoma,Appendiceal Cancer,B-Lymphoblastic Leukemia/Lymphoma,...,Thyroid Cancer,Tubular Adenoma of the Colon,UNKNOWN,Undifferentiated Embryonal Sarcoma of the Liver,Uterine Cancer,Uterine Sarcoma,Vaginal Cancer,Vulvar Carcinoma,Vulvar/Vaginal Cancer,Wilms Tumor
0,GENIE-CHOP-C1003065,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,GENIE-CHOP-C1003188,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,GENIE-CHOP-C1003434,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,GENIE-CHOP-C1003557,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,GENIE-CHOP-C1003680,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
df_cs.describe()

,Adenocarcinoma In Situ,Adrenal Gland Cancer,Adrenocortical Adenoma,Adrenocortical Carcinoma,Ampullary Cancer,Anal Cancer,Angiomatoid Fibrous Histiocytoma,Appendiceal Cancer,B-Lymphoblastic Leukemia/Lymphoma,Biliary Tract Cancer,...,Thyroid Cancer,Tubular Adenoma of the Colon,UNKNOWN,Undifferentiated Embryonal Sarcoma of the Liver,Uterine Cancer,Uterine Sarcoma,Vaginal Cancer,Vulvar Carcinoma,Vulvar/Vaginal Cancer,Wilms Tumor
count,146565.000000,146565.000000,146565.000000,146565.000000,146565.000000,146565.000000,146565.000000,146565.000000,146565.000000,146565.000000,...,146565.000000,146565.000000,146565.000000,146565.000000,146565.000000,146565.000000,146565.000000,146565.000000,146565.000000,146565.000000
mean,0.000027,0.000007,0.000191,0.001603,0.002333,0.002347,0.000096,0.004735,0.005151,0.000014,...,0.014799,0.000246,0.004988,0.000055,0.000020,0.004626,0.001098,0.000020,0.000020,0.001126
std,0.005224,0.002612,0.013821,0.040010,0.048249,0.048390,0.009773,0.068649,0.071588,0.003694,...,0.120748,0.015671,0.070446,0.007388,0.004524,0.067857,0.033125,0.004524,0.004524,0.033534
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [50]:
df_cs.columns[-50:]

Index(['Miscellaneous Neuroepithelial Tumor', 'Myelodysplastic Syndromes',
       'Myelodysplastic/Myeloproliferative Neoplasms',
       'Myeloid Neoplasms with Germ Line Predisposition',
       'Myeloproliferative Neoplasms', 'Myofibromatosis', 'Nerve Sheath Tumor',
       'Non-Hodgkin Lymphoma', 'Non-Small Cell Lung Cancer', 'Other Cancer',
       'Ovarian Cancer', 'Ovarian/Fallopian Tube Cancer', 'Pancreatic Cancer',
       'Parathyroid Cancer', 'Penile Cancer', 'Peripheral Nervous System',
       'Peripheral Nervous System Cancer', 'Peritoneal Cancer',
       'Peritoneal Cancer, NOS', 'Pheochromocytoma', 'Pineal Tumor',
       'Posttransplant Lymphoproliferative Disorders',
       'Primary CNS Melanocytic Tumors', 'Prostate Cancer',
       'Renal Cell Carcinoma', 'Renal Neuroendocrine Tumor', 'Retinoblastoma',
       'Rhabdoid Cancer', 'Salivary Gland Cancer', 'Sellar Tumor',
       'Sex Cord Stromal Tumor', 'Skin Cancer', 'Skin Cancer, Non-Melanoma',
       'Small Bowel Cancer', '

In [21]:
patient_cancer_type = pd.DataFrame(genie_processed["clinical_sample"].groupby("PATIENT_ID")["CANCER_TYPE"].unique())

In [22]:
patient_cancer_type.head()

,CANCER_TYPE
PATIENT_ID,
GENIE-CHOP-C1003065,[Embryonal Tumor]
GENIE-CHOP-C1003188,[Embryonal Tumor]
GENIE-CHOP-C1003434,[Glioma]
GENIE-CHOP-C1003557,[Glioma]
GENIE-CHOP-C1003680,[Glioma]


In [23]:
patient_cancer_type.reset_index(inplace=True)

In [24]:
patient_cancer_type.head()

,PATIENT_ID,CANCER_TYPE
0,GENIE-CHOP-C1003065,[Embryonal Tumor]
1,GENIE-CHOP-C1003188,[Embryonal Tumor]
2,GENIE-CHOP-C1003434,[Glioma]
3,GENIE-CHOP-C1003557,[Glioma]
4,GENIE-CHOP-C1003680,[Glioma]


In [25]:
patient_cancer_type["NUMBER_CANCER_TYPE"] = patient_cancer_type["CANCER_TYPE"].apply(lambda x: len(x))

In [26]:
patient_cancer_type.head()

,PATIENT_ID,CANCER_TYPE,NUMBER_CANCER_TYPE
0,GENIE-CHOP-C1003065,[Embryonal Tumor],1
1,GENIE-CHOP-C1003188,[Embryonal Tumor],1
2,GENIE-CHOP-C1003434,[Glioma],1
3,GENIE-CHOP-C1003557,[Glioma],1
4,GENIE-CHOP-C1003680,[Glioma],1


In [29]:
patient_cancer_type["NUMBER_CANCER_TYPE"].value_counts()

1    143673
2      2724
3       150
4        16
5         2
Name: NUMBER_CANCER_TYPE, dtype: int64

In [ ]:
del(patient_cancer_type["NUMBER_CANCER_TYPE"])

In [38]:
pd.pivot_table(patient_cancer_type,
               index = "PATIENT_ID", columns = "CANCER_TYPE",
               aggfunc = "count")

TypeError: unhashable type: 'numpy.ndarray'

In [33]:
df = (patient_cancer_type.set_index('PATIENT_ID').
      CANCER_TYPE.str.split(',\s*', expand=True).stack().reset_index()
      .pivot_table(index=['PATIENT_ID'], columns=0, fill_value=0, aggfunc='size'))

In [40]:
patient_cancer_type["CANCER_TYPE"] = patient_cancer_type["CANCER_TYPE"].apply(lambda x: list(x))

In [42]:
patient_cancer_type["CANCER_TYPE"].apply(lambda x: x.split(',\s*', expand=True))

AttributeError: 'list' object has no attribute 'split'